In [17]:
import numpy as np
import re
import operator

corpus = open('corpus.txt')
corpus = corpus.read()

regex = r"([a-zA-Z]+)\/([A-Z]+)|([^a-zA-Z]+)\/([^a-zA-Z])"
matches = re.finditer(regex, corpus)
words_tagged = []
# words_tagged = [match.group(3).lower(), match.group(4) for match in pattern if match.group(1) == None or match.group(2) ==  None else match.group(1).lower(), match.group(2)]
for match in matches:
    if match.group(1) == None or match.group(2) ==  None:
        words_tagged.append((match.group(3).lower(), match.group(4)))
    else:
        words_tagged.append((match.group(1).lower(), match.group(2)))

words_unique = np.unique([pair[0] for pair in words_tagged])

word_tag_dictionary = {}
for word in words_unique:
    word_tags = []
    for pair in words_tagged:
        if pair[0] == word:
            word_tags.append(pair[1])
    
    unique, counts = np.unique(word_tags, return_counts=True)
    
    word_tag_dictionary[word] = dict(zip(unique, counts))

unique, counts = np.unique([pair[1] for pair in words_tagged], return_counts=True)
tag_counts = dict(zip(unique, counts))

# for tag in tag_counts:
#     print(tag_counts[tag])

def tag(word):
    probability = {}
    
    if word not in word_tag_dictionary:
        return ('UNK', 1)

    for tag in word_tag_dictionary[word]:
        
        likelihood = word_tag_dictionary[word][tag] / tag_counts[tag]
        prior = tag_counts[tag] / len(words_tagged)

        probability[tag] = likelihood * prior
    
    return sorted(probability.items(), key=operator.itemgetter(1), reverse=True)[0]

test_string = open('test.txt')
test_string = test_string.read()

test_words = re.findall(r"[a-zA-Z]+|[^a-zA-Z\s]", test_string.lower())

print("Test: ", test_words)

string_result = ""
result = []
for word in test_words:
    result_tag = tag(word)[0]
    string_result += word +'/'+ result_tag +" "
    result.append((word, result_tag))

print("Results\n", string_result)
print(result)

ground_truth = open('truth.txt')
ground_truth = ground_truth.read()

truth_matches = re.finditer(regex, ground_truth)
ground_truth = []
for match in truth_matches:
    if match.group(1) == None or match.group(2) ==  None:
        ground_truth.append((match.group(3).lower(), match.group(4)))
    else:
        ground_truth.append((match.group(1).lower(), match.group(2)))

print("Ground truth\n", ground_truth)

accuracy = 0
for r, t in zip(result, ground_truth):
    print(r, t)
    if r == t:
        accuracy += 1

print(len(result), len(ground_truth))
print("accuracy: " + str(accuracy / len(result) * 100) + "%")

Test:  ['ted', 'terkenal', 'di', 'amerika', 'serikat', 'dalam', 'waktu', '4', 'tahun', '.', 'ted', 'mengatakan', 'bahwa', 'kalian', 'brengsek', '.', 'aku', 'mau', 'mengambil', 'salah', 'satu', 'trophy', 'untuk', 'terlihat', 'ganteng', '.', 'kalian', 'semua', 'sama', 'seperti', 'profesor', 'psikopat', '.', 'mobilnya', 'tidak', 'bisa', 'cepat', 'karena', 'kalian', 'tidak', 'ganteng', '.']
Results
 ted/NNP terkenal/VB di/IN amerika/NNP serikat/NNP dalam/IN waktu/NN 4/UNK tahun/NN ./UNK ted/NNP mengatakan/VB bahwa/SC kalian/PRP brengsek/JJ ./UNK aku/PRP mau/RB mengambil/VB salah/NEG satu/CD trophy/FW untuk/SC terlihat/VB ganteng/JJ ./UNK kalian/PRP semua/CD sama/JJ seperti/SC profesor/NN psikopat/NNP ./UNK mobilnya/NN tidak/NEG bisa/MD cepat/JJ karena/SC kalian/PRP tidak/NEG ganteng/JJ ./UNK 
[('ted', 'NNP'), ('terkenal', 'VB'), ('di', 'IN'), ('amerika', 'NNP'), ('serikat', 'NNP'), ('dalam', 'IN'), ('waktu', 'NN'), ('4', 'UNK'), ('tahun', 'NN'), ('.', 'UNK'), ('ted', 'NNP'), ('mengatakan',